In [5]:
import random
import numpy as np
import pandas as pd
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
from torch import optim

In [6]:
"""
Generating the training data
Taking a random list of 20 words with this website : https://www.randomlists.com/random-words?dup=false&qty=20
"""
word_list = ['nice', 'toothpaste', 'halting', 'deafening', 'structure', 'soak', 'omniscient', 'found', 'authority',
             'crabby', 'blush', 'dreary', 'childlike', 'nippy', 'roll', 'highfalutin', 'wing', 'hole', 'nonchalant',
             'plant']

window_length = max([len(w) for w in word_list]) + 1
reformat_to_window = lambda w: w + ' ' * (window_length - len(w))
word_list_formated = [reformat_to_window(w) for w in word_list]


In [7]:
def shuffle_letters(word, shuffle_letterse=5):
    """ Recursive function that exchange two letters at each iteration (shuffle)"""
    if type(word) is not list:
        word = list(word)
    if shuffle == 0:
        return ''.join(word)  # word==shuffle_letters(word,shuffle=0) -> TRUE
    i, j = random.randint(0, len(word) - 1), random.randint(0, len(word) - 1)
    word[i], word[j] = word[j], word[i]
    return shuffle_letters(word, shuffle - 1)


In [15]:
size_of_data = 10000
train_data = [reformat_to_window(shuffle_letters(random.choice(word_list), random.randint(1, 3) * random.randint(0, 1))) \
              for _ in range(0, size_of_data)]

## Output Encoding


In [26]:
"""
In order to train the model need a supervised train_data. Let's encode the output as a vector of binaries where 1 
in i ith position if the model recognize the i ith word of the list. And declare as a non-sense if its recognize two 
different words. 
"""
nn_output = np.zeros(len(word_list))

recognize = lambda w: np.array([1 if w == w_f else 0 for w_f in word_list_formated])
train_y = [recognize(w) for w in train_data]

# Print proportion of correct words after shuffle
#print(sum([sum(x) for x in train_y]) / len(train_data))  

decode_output = lambda o: 'Not recognize' if np.sum(o) >= 2 or np.sum(o) == 0 else word_list[o.tolist().index(1)]
nn_output = train_y[1]
#print(nn_output)
#print(decode_output(nn_output))


[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
Not recognize


## Input Encoding

In [33]:
"""
Encode the input look at this https://stackoverflow.com/questions/7396849/convert-binary-to-ascii-and-vice-versa
"""

# Here is a little demonstration that we need 5 binaries to distinguish the 26 minus letter
alpha = 'abcdefghijklmnopqrstuvwxyz'
binaries = [bin(ord(letter_min))[-5:] for letter_min in list(alpha)]
# print(len(binaries), len(list(set(binaries)))) # suppress the doublon in the second list

nn_encode = np.zeros(window_length * 5)


def encode_window(word=word_list_formated[0]):
    # print(word)
    global nn_encode
    word_binaries = [bin(ord(l))[-5:] for l in list(word)]
    word_binaries = ''.join(word_binaries)
    word_binaries = [int(b) for b in list(word_binaries)]
    # print(word_binaries)
    return np.array(word_binaries)


def decode_window(input_binaries_list):
    word_binaries = input_binaries_list.tolist()
    word_binaries = ''.join([str(b) for b in input_binaries_list])
    word_binaries = ['011' + word_binaries[i:i + 5] for i in range(0, len(word_binaries), 5)]
    # print(word_binaries)
    return ''.join([chr(int(x, 2)) for x in word_binaries])


nn_encode = encode_window(word=word_list_formated[3])
decode_window(nn_encode)


'deafening```'

## Neural Networks

In [34]:
# Initiate 2 hidden layers 
nn_hidden_layer_1 = np.zeros(10)
nn_hidden_layer_2 = np.zeros(10)

# Print architecture of NN
#print(nn_encode.size, nn_hidden_layer_1.size, nn_output.size)
""" Why 60-10-20? 
Because in Input, we encode each input as length of 12 -> "blink```````" 
and for each character in input, we encode it as binary of length 5, so 5*12 = 60 input neurons
in Output we get 20 neurons and each neuron is 0/1. It is map decision of our Recognition Word list created 
at very beginning of task - word_list. Each binary output means whether NN could recognize given input as any member 
of word_list
"""

# Link the layer by neurons:
sigmoid = lambda x: 1 / (1 + np.exp(-x))
sigmoid_ = lambda y: y * (1 - y)

# correct solution:
def softmax(x):
    """Compute softmax values for each sets of scores in x."""
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum(axis=0) # only difference
add_row_ones=lambda x : np.array([*x,np.ones(x.shape[1])])
class NeuralNetwork:
    def __init__(self, input_layer=nn_encode, l_hidden_layers=[nn_hidden_layer_1], output_layer=nn_output):
        self.layers = np.array([input_layer, *l_hidden_layers, output_layer])

        'Layers: 60-10-20 '
        #print(len(self.layers[0]), len(self.layers[1]), len(self.layers[2]))

        L = []
        for i in range(0, self.layers.size - 1):  # !!!! L for weigth corespond to L-1 for Layers
            weight = np.random.rand(self.layers[i].size + 1, self.layers[i + 1].size)

            weight[-1] = -1 * weight[-1]  # Change the sign of the biases
            L.append(weight)  # self.weights[L][k][l] : L : Layer  ; k neurone ; l edge
        self.weights = np.array(L)

        'Weights: (61, 10) and (11, 20)'
        #print(self.weights[0].shape, self.weights[1].shape)


    def input_word(self, word):
        self.layers[0] = encode_window(word)

    def input_decode(self):
        return (decode_window(self.layers[0]))

    def output(self):
        return (self.layers[-1])

    def decode_output(self):
        return decode_output((self.layers[-1] >= 0.5)) * 1

    # Inspect this
    def feedfoward(self):
        """Sigmoid  -> 61 * (61, 10) = [10]
           Softmax -> 11 * (11, 20) = [20]"""
        self.layers[1] = sigmoid(np.dot(np.array([*self.layers[0], 1]), self.weights[0]))
        self.layers[2] = softmax(np.dot(np.array([*self.layers[1], 1]), self.weights[1]))
        #print(self.layers[1])
        #print(self.layers[2])

    # Inspect this
    def backpropagation(self, desiered_output, eta):
        error_vector=self.output()-desiered_output
        cost = np.sum(error_vector*error_vector)
        for L in range(0,self.layers.size-1):
            L=self.layers.size-2-L # browse the list backwards
            delta=sigmoid_(self.layers[L+1])*error_vector #
            mat_delta=np.matmul(np.array([*self.layers[L],1]).reshape(self.layers[L].size+1,1),delta.reshape(1,delta.size)) ## Adding 1 for
            self.weights[L]=self.weights[L]-eta*2*mat_delta ## Actualize the weight
            error_vector=np.matmul(self.weights[L][0:-1],error_vector.T) ## Propagate the error (without biases)
        return cost



60 10 20


In [12]:
nn = NeuralNetwork(l_hidden_layers=[nn_hidden_layer_1])

In [37]:
print(len(train_data))
print(len(train_y))

10000
10000


In [14]:
p = 0
for e in range(100):
    err = 0 
    for i, w in enumerate(train_data):
        nn.input_word(w)
        nn.feedfoward()
        cost = nn.backpropagation(desiered_output=train_y[i], eta=0.0001)
        err += cost #where to put cost and how to construct it?
    print(err)



5609.3560978766645
5595.940997676815
5584.848547342994
5575.561435945866
5567.690598177439
5560.938022304365
5555.071650953548
5549.907888954162
5545.2990746195055
5541.124281636536
5537.282415161161
5533.686945994576
5530.261897168262
5526.93892079646
5523.655514453243
5520.354624452623
5516.985999320526
5513.509511230338
5509.899997024918
5506.151930198471
5502.281090894802
5498.32059662025
5494.310402381899
5490.28144097462
5486.238216739387
5482.147528807604
5477.94152645899
5473.537172272387
5468.868053282136
5463.922967418781
5458.78084482948
5453.619505098962
5448.677731929759
5444.1837509430625
5440.2913981821175
5437.051011805811
5434.415105874116
5432.267085187406
5430.457667300169
5428.83619994081
5427.272950861118
5425.675530313835
5424.002090484639
5422.268530197085
5420.541940678967
5418.915496257137
5417.476116214225
5416.287556572437
5415.3951634655095
5414.835328298784
5414.634995011918
5414.798384240445
5415.287244027037
5416.018288483856
5416.888394025311
5417.804729

In [ ]:
word_list_counts = 0
word_list_result = 0
not_r = 0
for word in mytest:
    nn.input_word(reformat_to_window(word))
    nn.feedfoward()
    if word in word_list:
        word_list_counts += 1
        if nn.decode_output() == word:
            word_list_result += 1
    else: 
        if nn.decode_output() == "Not recognize":
            not_r += 1
        else:
            print(word,nn.decode_output())
        
print(word_list_counts, word_list_result)
print(not_r)


In [3]:
test_size = 100
mytest = [shuffle_letters(random.choice(word_list), random.randint(1, 3) * random.randint(0, 1)) \
              for _ in range(0, test_size)]


NameError: name 'shuffle_letters' is not defined

In [2]:
# set=7000, 200 epoch , 0.1 eta ==> 960
#nn = best_nn
best_nn = nn
first = best_nn.weights[0]
second = best_nn.weights[1]

NameError: name 'nn' is not defined

In [1]:
word = "wing"
nn.input_word(reformat_to_window(word))
nn.feedfoward()
if nn.decode_output() == word:
    print("True")
else:
    print("Not Recognize")

NameError: name 'nn' is not defined

In [414]:
word_list = ['nice', 'toothpaste', 'halting', 'deafening', 'structure', 'soak', 'omniscient', 'found', 'authority',
             'crabby', 'blush', 'dreary', 'childlike', 'nippy', 'roll', 'highfalutin', 'wing', 'hole', 'nonchalant',
             'plant']

First weight
[[-1.06411079e+00  5.85941250e+00 -1.48025419e+00 -1.81013048e+00
  -7.60118582e+00 -3.45246558e+00  5.97176610e+00 -6.93783774e+00
  -1.38352394e+00 -8.07739895e+00]
 [-5.36565257e+00 -4.43462755e+00 -1.30858192e+01 -5.84733269e+00
  -9.56342711e+00 -2.70067172e+00  3.21027724e+00 -7.04123155e+00
   1.65328489e+00  1.17913272e+01]
 [ 2.92470522e+00 -3.34267074e+00 -2.29767548e+00 -3.01891242e+00
  -2.96046542e+00  6.16015269e+00 -8.54164095e+00 -8.26144149e+00
   1.22830486e+00  8.80162701e-01]
 [ 4.22470535e+00  1.75435223e+00  5.59009686e+00  2.61166666e+00
  -5.68107582e+00  4.09581951e-01  5.94357571e+00 -2.03348032e+00
   2.84151232e+00 -3.51638337e-01]
 [-1.82572246e+00 -6.40788287e+00 -8.38099769e+00  9.25455627e-01
  -7.51240732e+00 -6.40097901e-01 -2.04556755e+01 -5.35632882e+00
  -5.89177457e+00  6.15166516e+00]
 [-1.96978443e-01 -8.16913594e+00 -4.39849169e+00  4.44040225e+00
   2.34765022e+00  5.71573575e+00 -3.28667127e+00 -1.72800275e+00
  -4.63543643e+00  1.03172795e+00]
 [ 2.58744453e+00 -8.56176782e-01  3.58374415e+00 -2.74429499e+00
  -7.04386039e+00 -3.51649840e+00  5.50915820e+00 -7.28911870e+00
   2.62415965e+00 -3.62629249e+00]
 [ 4.39787995e+00 -3.03096208e+00  5.13196971e+00 -7.77897997e+00
  -2.27521683e+00 -2.25971588e+00 -4.18509232e+00 -3.66050083e+00
  -1.18874414e+00  2.50951861e+00]
 [ 2.23637707e-01 -9.05231786e+00  2.97189384e+00 -4.31780624e-01
   4.25520146e+00 -1.09868885e+00 -9.02036085e-01 -3.06566338e+00
  -6.16667401e+00  8.41567374e+00]
 [-8.07850294e-01  5.12315416e-01  5.76033396e+00 -6.83340446e+00
   4.76367312e+00 -4.37604348e+00  8.76477759e+00  3.60300451e+00
   4.87159767e+00 -1.52370352e+00]
 [ 8.09312139e+00  2.31463253e+00  1.50415439e-01 -2.88755813e+00
  -1.20297449e+00 -3.16690079e+00 -6.72631879e+00 -2.51160581e+00
  -3.37837186e+00  4.65782346e+00]
 [ 3.82366037e+00 -4.66711278e+00 -3.98080589e+00 -1.86663645e+00
  -7.54204559e+00 -3.20840011e+00 -3.67608381e+00 -6.19816080e-01
  -2.62990100e+00  3.98863980e+00]
 [ 2.18751974e+00 -1.53207632e+01 -1.02236263e+00 -4.91633852e+00
   4.33601328e+00  6.38031659e-01  1.27764078e+00  8.45649549e+00
  -5.94790502e+00  7.14059324e+00]
 [ 6.42600189e+00 -5.30711378e+00 -3.74801689e+00 -4.66069013e+00
  -3.05129998e+00 -8.56723398e+00  1.72462786e+00 -3.55357284e+00
   5.39447605e+00 -2.38845629e+00]
 [-4.57634173e-01 -1.01421092e+01  1.81949444e+00  1.06701463e+00
  -1.65511486e+00  7.41692981e+00 -6.37604875e+00 -4.54137646e+00
  -5.91132896e+00  3.57073404e+00]
 [ 1.98374298e+00  6.68285103e+00  2.72280799e+00 -3.25113643e+00
  -5.07533363e+00 -5.66010491e+00 -7.50955923e+00  2.58731444e+00
  -4.76127789e+00  8.56597469e+00]
 [-5.13365631e+00 -8.02557159e+00  4.21968138e-01  4.23129239e-01
  -1.23612061e+00 -2.84570421e+00 -9.06762317e+00 -2.74636497e+00
  -8.91218727e+00  9.63066402e+00]
 [ 1.90732956e+00 -3.99408036e+00 -4.30646524e+00  6.63552976e-02
  -7.57094661e+00 -1.02053296e+01  3.66236391e+00 -5.72467797e+00
  -3.38205397e+00 -6.24344755e+00]
 [ 2.80231525e+00 -2.44244957e+00  3.09069994e+00 -4.15950619e-01
  -6.28283198e+00 -4.98580346e+00 -7.59731045e+00 -5.83959548e+00
   4.18763728e-01 -6.62859339e+00]
 [ 1.32739814e+00 -2.64774099e+00 -7.34165727e+00 -6.01713407e+00
  -5.95166193e+00 -7.94400608e+00 -9.05754569e-01 -8.15153394e+00
   9.72947912e-01  4.67220613e-01]
 [-8.08981627e+00  3.28236135e-01 -6.28718161e+00 -3.40761145e+00
  -8.97028134e-01 -3.18788163e+00 -6.96480475e+00 -3.06779163e+00
   5.57344212e+00  4.26575088e+00]
 [-4.57930313e-02 -4.97709375e-01 -3.22580400e+00 -7.31081884e+00
  -5.17213710e+00 -1.07235153e+00 -2.58416605e+00  3.48415807e+00
   5.58867086e-01  8.81172843e+00]
 [-7.61481940e+00 -5.91873536e+00 -2.34412406e+00 -1.86408383e+00
   1.71305120e-01 -3.13456326e+00 -5.43604222e+00 -1.98620342e+00
  -1.17438648e+01 -4.57307700e+00]
 [ 1.27298199e+00 -3.55079818e+00 -8.07769736e+00 -3.02890144e-01
   5.89130119e+00  1.50791318e+00 -4.32200466e+00 -8.47733485e-01
  -2.87082763e-01  1.05685754e+01]
 [-3.26370567e+00  7.82391800e+00 -5.61508394e+00 -7.03828718e+00
  -1.17689012e+00 -9.11122595e+00 -5.00475627e+00  1.73947681e+00
  -2.08600284e+00  1.40330652e+00]
 [ 3.55292465e-01 -1.15697929e+00 -1.58995045e+00  1.24657493e+00
   5.02146230e+00  1.06075805e+00 -5.41708742e-01  1.74167993e+00
  -2.03823412e+00  2.85078710e+00]
 [ 4.42092826e-01  4.01211879e+00 -3.30840718e+00  1.49827684e+00
  -3.62505817e+00  2.37555653e+00 -1.39796667e+00 -1.09516791e+00
  -9.11228267e+00 -1.28057344e+00]
 [-7.97469123e+00 -8.73337522e-01 -6.90102281e+00  1.69930925e+00
  -9.67659417e+00 -1.10122037e+00 -1.19757387e+01 -1.88780054e+00
  -9.41828683e+00 -1.27572095e+00]
 [-1.03970176e+01  6.59114871e-01 -1.73668547e+00 -2.37655967e+00
  -1.41395484e+00 -2.25059471e+00 -1.76734123e+00  4.36275801e+00
  -9.79818828e+00  1.48809737e+00]
 [-3.96887318e+00 -8.57948816e+00 -4.41383735e+00 -2.76134483e+00
   2.36346205e-01 -5.10220090e-01 -5.35163188e+00 -5.56287351e+00
  -6.28558840e-01  3.20693167e+00]
 [ 3.10376223e+00  3.02884454e+00 -1.87715780e+00 -1.53428376e-01
  -3.70018829e+00 -5.64465790e-02 -7.21192107e-01 -4.72847517e+00
  -4.39752701e+00 -2.60697219e+00]
 [-1.89747909e+00  6.16890301e+00 -3.44888192e+00 -1.62369593e+00
   3.03096637e-01 -9.18009038e-02 -1.64185009e+00 -2.00701302e-01
   1.82945312e+00 -7.50807768e-01]
 [-4.20234656e+00 -6.35522764e+00 -5.03287263e+00 -2.02368604e+00
   3.18645074e+00 -3.42540181e-01 -5.61879417e+00 -1.09527368e+00
  -6.28423815e+00  1.19274307e+01]
 [-5.81956559e+00 -6.16883787e+00 -3.16945853e+00 -5.37194113e+00
  -4.69304174e+00 -7.81078030e-02 -1.87311156e+00  1.92709191e+00
  -9.08010009e+00 -6.01883456e-01]
 [ 6.55303710e+00  5.37350133e+00 -5.10102030e-01  2.35306768e+00
   4.15777845e-01 -5.63841898e-01  3.29286588e+00  3.11265545e+00
   4.29290705e+00  6.90382811e+00]
 [ 1.14337218e+00  8.85286977e-02  2.38186736e-01  3.83655439e-01
   3.62632480e+00  8.51980034e-02  2.17092979e+00 -7.46289533e-01
   2.70060785e+00  5.84428919e+00]
 [-1.10433665e+00  1.96252371e+00 -1.83372998e+00  2.23684517e+00
  -4.78680905e+00  2.64241660e-01 -3.50933690e-01 -1.23852308e+00
  -9.47309985e+00  3.28486389e-01]
 [-8.40135476e+00 -5.25528239e+00 -3.84259860e+00 -1.46565414e+00
  -8.08322930e-01 -6.11574312e-01 -2.51786798e+00  4.99764754e-01
  -1.07728975e+00  1.83009933e+00]
 [ 1.02971646e+01  3.25533648e+00  1.20482030e+00  3.45350579e+00
  -4.78207239e+00 -5.85564476e-01  3.69295445e+00 -1.20157830e+00
  -9.75175578e+00 -2.76564008e-01]
 [-1.01171481e+01 -1.10728188e+01 -4.31578544e-01 -9.09824422e-01
  -4.99330901e+00 -2.60724384e-01 -5.63622644e-01 -3.53423916e+00
   9.09875224e+00  2.72146206e+00]
 [-6.24212148e+00 -6.01680924e+00  1.21447868e-01  4.22581409e-01
   7.83793855e+00 -9.81577741e-02  1.89127478e+00  3.22040416e+00
  -4.72340854e-01  3.10000235e+00]
 [-4.63117841e+00 -1.08436854e+00 -1.65301674e+00 -5.12104669e+00
  -1.66267498e+00 -4.20966315e-01 -3.25273807e+00  1.09446717e+00
  -1.79609861e+00  2.37718547e-01]
 [ 4.02274390e+00  4.95442917e+00 -1.16018486e+00  1.12216778e+00
  -3.36259975e+00 -7.11537864e-01  1.00686453e+00 -3.55403614e+00
  -4.11860499e-01  1.03100238e+00]
 [-6.51929309e+00  3.05640808e+00 -2.13130787e+00 -6.05841010e-01
  -2.10389842e+00 -3.05257441e-01 -8.33990234e-01 -1.15124587e+00
   2.47751666e+00 -1.70165586e+00]
 [-1.81893619e+00  5.76815865e+00 -2.49082528e+00  1.56420032e+00
  -3.03673722e+00 -1.62666833e-01 -1.35059261e+00 -1.22260611e-03
  -1.00086067e+01 -4.45416985e+00]
 [-3.21936654e+00  2.85212162e-01 -8.75944662e-01  7.67857772e-02
  -2.30769149e-01  1.80278379e-01 -1.20096068e+00  7.59005712e-02
   2.81977535e+00  3.12530705e-01]
 [-2.72739694e+00 -3.77700086e-01 -1.92483677e+00  8.59360139e-01
   3.05555895e+00 -4.80936887e-01 -3.59764386e+00 -1.87639598e-01
  -6.92959185e+00  9.77284016e-01]
 [ 2.92182753e+00 -1.58884370e-01  6.46979323e-02 -1.04605086e+00
  -1.79649427e+00 -3.36803374e-01  1.63321187e+00  4.47524972e-01
   3.24426679e+00 -5.20176012e+00]
 [-2.66038157e+00  3.83384989e-01 -1.75645114e+00 -5.74277487e-02
  -9.60598232e-01  5.94611313e-01 -2.95422026e+00  7.52513460e-01
  -3.86153243e+00 -1.56653399e+00]
 [ 2.14451746e+00 -3.63784926e-01 -6.96944528e-01  6.41660135e-01
   1.31514209e+00  3.35866345e-01  1.24425329e+00 -7.01489564e-02
   5.32797608e-01  2.33180396e+00]
 [ 9.51791881e-01  5.25387098e-01  1.76621686e-01  4.91875275e-01
  -2.36658256e-01  6.76124265e-01 -7.52489293e-02  8.03056055e-02
   5.42498321e-01  7.26013698e-03]
 [-2.45204219e-01  5.24607914e-01  2.91910104e-01  9.63957962e-01
   4.65167086e+00  2.13240247e-01  1.12150872e-01  3.10118186e-01
   6.80692933e-02  7.30260433e+00]
 [-8.82381716e-02  2.88136883e-01  5.92110793e-01  2.18205603e-01
  -3.19210960e-01  1.42309627e-01  2.27154107e-01  2.04270369e-02
  -4.97796517e-01 -2.08300351e+00]
 [ 4.57295834e-01 -1.26699000e-01  3.32646993e-01  8.03716740e-01
   2.40899061e+00  4.19563042e-02 -5.65755479e-02  2.33465671e-01
  -2.30015354e-01  1.40171632e+00]
 [ 2.68580013e-01  1.48441376e-01  1.03901223e-01  1.71054798e-02
  -2.50460398e+00  7.92799117e-01  6.55617203e-01  8.32543171e-01
  -2.73471783e-02 -1.28487949e+00]
 [ 6.33455235e-01  6.62500291e-01  7.94167018e-02  2.16020533e-01
   9.50625212e-01  7.89708759e-02  2.47798570e-02  2.74725306e-01
   6.75077631e-01  2.04381500e-01]
 [ 9.95652420e-01  9.60023853e-01  3.65920657e-01  4.98819509e-01
   8.64760129e-01  6.05527556e-01  9.49230567e-01  3.11320849e-01
   1.75749137e-01  5.34817076e-01]
 [ 6.04456802e-01  5.49743074e-01  1.91516309e-01  4.23855623e-01
   4.85519466e-01  6.50005200e-01  5.87003146e-01  8.18018009e-02
   8.43115415e-01  1.56651557e-01]
 [ 1.30944995e-02  6.96688740e-02  3.71061936e-01  9.92171006e-01
   7.79629270e-01  5.15228661e-01  5.50679184e-01  6.61699764e-01
   8.93693769e-01  5.60157064e-01]
 [ 6.59708270e-02  6.34181307e-01  2.65644650e-01  9.02927058e-01
   4.40377613e-01  4.07255774e-01  6.71696994e-01  4.83144534e-01
   7.50793758e-01  8.81333316e-01]
 [-5.93960557e+00 -5.72380550e+00 -1.21608791e+01 -6.11352011e+00
  -5.87355418e+00 -7.51091286e+00 -7.74304982e+00 -7.30305239e+00
  -9.52950914e+00  7.69368178e+00]]

Second weight
[[-2.43957991e+00  6.68239308e-01 -5.34401382e-01 -4.14456314e+00
   5.89978784e-01 -4.04400228e+00 -2.49044693e+00 -3.03171357e+00
  -4.07102203e-01 -1.32337200e+00  1.20891091e+00 -1.04754763e+00
  -1.31005506e+00 -2.99404288e+00 -2.39387662e+00 -1.55947453e+00
  -3.54994739e-01 -3.77765556e+00 -1.03909338e+00 -3.33235377e+00]
 [-3.19439806e-01 -1.38601334e+00 -2.76850878e-01  1.25795345e+01
   9.72704024e+00 -9.46773624e-02 -2.93120733e-01  2.83210847e-01
  -3.70336219e-01 -4.37527043e-01 -2.83654598e+00 -2.97455562e-01
  -2.04913864e-01  1.22515697e+01 -4.96829102e-01 -3.74427205e-01
  -3.74977497e-01 -2.96183085e-01 -2.16702162e-02 -7.36327744e-01]
 [-9.87406500e-01  3.51173252e+00 -4.52994024e-01  1.13845748e-01
  -9.58074182e-01  3.03020168e+01 -4.38379176e-01  3.39089803e+01
  -2.69616362e-01  1.99785691e+00  1.77889951e+01  1.75150093e+00
  -3.16093910e-01 -2.60087533e-01  1.29014800e+01 -4.87831059e-01
   6.93615664e-02 -2.11269263e+00 -4.30142147e-01  2.21091587e-02]
 [-7.39715501e-03 -2.62593292e-01 -3.44081558e-03  4.35329200e-01
  -1.43383655e-01  3.41741136e-01 -1.59989652e-01 -9.64055821e-02
   1.69506822e-02  4.97651497e+00  3.91070549e-01 -1.07695596e+00
   9.61126337e+00 -1.56966871e-01  1.51018336e-01  1.76362677e+00
  -2.43762888e-01 -4.44794755e-02  4.25061283e-01 -3.21299401e-01]
 [-5.01297226e-01 -3.67763385e-01 -1.40212090e+00  2.92238740e-03
   2.77215158e-01 -7.80025767e-01 -2.96766404e-01  7.81531616e-02
   5.61749244e+00 -1.23876825e+00 -2.24305864e-02  4.17841550e+00
  -6.29907996e-01 -1.12902342e+00 -3.62762054e-01  8.51545023e+00
  -7.95855074e-01 -6.33128089e-01  2.90551251e-01 -8.50756281e-01]
 [ 5.28700712e-01  3.30170084e-01  4.62298722e-02  2.28990153e-01
  -1.02739110e-01 -1.91531552e-01  4.29712099e-01  3.43236137e-01
   1.69566630e-01  7.40823972e+00  2.75017294e-01  7.63225315e+00
  -1.59043658e+00  5.68409375e-03  8.14853189e-02  5.82097888e-02
  -2.02948387e-01  4.86237229e-01  4.05816149e-01  2.49570252e-01]
 [ 1.36965233e+01  1.28752786e+01  6.00799925e-01  3.19771698e+00
  -3.34841593e-01 -2.06081454e+00 -1.24103651e+00 -2.50501533e+00
   1.94549349e-01 -1.08946525e+00 -1.84281493e+00 -1.24108364e+00
   2.39264259e+00 -1.64990866e+00  1.32915312e+01  3.35186887e-01
  -2.46466190e-01  1.78492504e+01 -1.80316329e+00 -8.30085365e-01]
 [-4.64662760e-02  2.59568620e-01  7.40504021e+00  2.56935121e-02
   5.55909499e-02 -2.01085986e-01  1.52530960e-01  5.80878526e-01
   4.40223286e+00  3.63796572e-02 -4.59455246e-01 -3.67463021e-01
  -2.40395670e-01 -9.53349972e-01  1.14783819e-01 -2.11202410e+00
  -5.87423827e-01  2.69109782e-01  3.42543688e-01 -6.01309546e-01]
 [ 1.10511478e+01  6.01864792e+00 -3.95270664e-01 -2.12890289e+00
   1.34403199e-01 -1.59523264e+00  2.47194448e+01 -1.65592656e+00
  -3.59382644e-01  4.90100985e-03 -2.43447005e+00  6.82801606e-01
  -2.66378377e-01  6.41844798e+00 -2.02474132e+00 -6.80296696e-03
  -1.05935305e+00 -2.74261811e+00  2.83691274e+01 -1.66091860e+00]
 [-1.84365846e+01 -1.88705350e+01 -1.78086459e+01 -2.03974659e+01
  -1.90145560e+01 -1.86113863e+01 -1.84488194e+01 -1.82497606e+01
  -1.93600469e+01 -1.84066541e+01 -1.88708229e+01 -1.87756898e+01
  -1.84382711e+01 -1.66759885e+01 -1.85227140e+01 -1.76497573e+01
  -2.56733022e+01 -1.82952853e+01 -1.83458819e+01 -2.55044289e+01]
 [-2.55657212e+02 -2.56346255e+02 -2.56800914e+02 -2.53770400e+02
  -2.56197232e+02 -2.55020517e+02 -2.55274251e+02 -2.57694322e+02
  -2.55344312e+02 -2.55891667e+02 -2.56670666e+02 -2.55597998e+02
  -2.55860314e+02 -2.57604659e+02 -2.55719058e+02 -2.56593532e+02
  -2.49384260e+02 -2.55663643e+02 -2.57144789e+02 -2.48325727e+02]]